<a href="https://colab.research.google.com/github/yuvarbiv/DQN/blob/main/recreating_Baroch's_article.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ALL FUNCTIONS

In [ ]:
import random, math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

# ----- functions: -----
def cell_send_pulse(targets_loc, false_targets_loc, alpha, i, j):
  # return: 1==pulse, 0==no_pulse
  # later I'd like this function to return continuous value [0,1]
  if (i,j) in targets_loc:        return random.random() < P_ta
  if (i,j) in false_targets_loc:  return random.random() < alpha*P_ta
  return False

def agent_recieved_signal(d, lamda):
  # return: 1==pulse, 0==no_pulse
  # TODO:later I'd like this function to return continuous value [0,1]
  x = math.e**(-d/lamda)
  return random.random() < x

def cell_update_probabillity(prev_grid_val, frame, d, recieved):
  P_recieved__t0 = prev_grid_val
  NP_recieved__t0 = 1-P_recieved__t0
  # recieved
  numerator =             P_recieved__t0 * P_ta
  denumerator = numerator + NP_recieved__t0 * P_ta * alpha
  P_signal_recieved__t1 = numerator / denumerator if denumerator != 0 else 0 # Avoid division by zero
  # UNrecieved
  signal_strength = 1-math.e**(-d/lamda)
  numerator =           P_recieved__t0 * (NP_ta + P_ta*signal_strength)
  denumerator = numerator + NP_recieved__t0 * ((1-alpha*P_ta)+alpha*P_ta*signal_strength)
  P_signal_UNrecieved__t1 = numerator / denumerator if denumerator != 0 else 0 # Avoid division by zero

  if recieved:
    return P_signal_recieved__t1 # Using the calculated received probability
  else:
    return P_signal_UNrecieved__t1 # Using the calculated unreceived probability

def update_target_set(target_set, detect_img, i, j, threshold, new_grid_val, frame):
  if new_grid_val >= threshold and (i,j) not in [(t[0], t[1]) for t in target_set]:
    target_set.add((i,j,frame))
    detect_img[i, j] = frame  # reflect into detection image
    pairs,frame = print_targets_found(target_set)

def print_targets_found(target_tuples):
  pairs = [f"({i}),({j})" for i,j, _ in target_tuples]
  steps = [steps for _, _, steps in target_tuples]
  return pairs,steps

def update_cell(cell_by_agent,frame, grid, i, j):
  new_val = np.mean(cell_by_agent)
  grid[i][j] = new_val
# --------------------------------------------


#1 RUN + SIMULATION

In [ ]:

# ----- your model bits (edit as needed) -----
seed = 41
random.seed(seed)
np.random.seed(seed)

alpha = 0.5
P_ta = 1; NP_ta = 1 - P_ta
nof_iterations = 40
GSize = 50
lamda = GSize*2
nof_agents = 3
nof_targets = 3
nof_false_targets = 3
threshold = 0.9
size_of_markers = 20

# ----- initialize datasets -----
initial_grid_value = nof_targets/(GSize**2)
grid = np.full((GSize, GSize), initial_grid_value, dtype=float)
agents_loc = random.sample([(i,j) for i in range(GSize) for j in range(GSize)], nof_agents)
targets_loc = random.sample([(i,j) for i in range(GSize) for j in range(GSize)], nof_targets)
available_locs = [(i,j) for i in range(GSize) for j in range(GSize) if (i,j) not in targets_loc]
false_targets_loc = random.sample(available_locs, nof_false_targets)
print(grid)
print(agents_loc)
print(targets_loc)
print(false_targets_loc)

# ----- Simulation Initialization -----
# A 2D image to visualize detections (instead of trying imshow on a set)
detect_img = np.zeros_like(grid, dtype=float)
target_set = set()

# ---- single figure, two axes ----
fig, (Gax, Tax, Bax) = plt.subplots(1, 3, figsize=(20, 8))
Gim = Gax.imshow(grid, cmap='viridis', vmin=0, vmax=1)
Gax.set_title('Probabilities'); fig.colorbar(Gim, ax=Gax)
Tim = Tax.imshow(detect_img, cmap='viridis', vmin=0, vmax=nof_iterations)
Tax.set_title('Locations of Participants'); fig.colorbar(Tim, ax=Tax)
# bar chart axis setup
Bax.set_title('Detections by cell')
Bax.set_ylabel('Frame of first detection')
Bax.set_ylim(0, nof_iterations)
plt.tight_layout()

# draw static markers once
ax = [c[1] for c in agents_loc]; ay = [c[0] for c in agents_loc]
# Gax.scatter(ax, ay, marker='o', s=size_of_markers, color='black', label='Agent')
Tax.scatter(ax, ay, marker='o', s=size_of_markers, color='black', label='Agent')
tx = [c[1] for c in targets_loc]; ty = [c[0] for c in targets_loc]
# Gax.scatter(tx, ty, marker='X', s=size_of_markers, color='white', label='Targets')
Tax.scatter(tx, ty, marker='X', s=size_of_markers, color='white', label='Targets')
fx = [c[1] for c in false_targets_loc]; fy = [c[0] for c in false_targets_loc]
# Gax.scatter(fx, fy, marker='X', s=size_of_markers, color='red', label='False Targets')
Tax.scatter(fx, fy, marker='X', s=size_of_markers, color='red', label='False Targets')
Gax.legend(loc="upper center", bbox_to_anchor=(0.5, -0.1), ncol=3)
# List to hold text objects for detections
texts = []

# ----- main loop -----
def update(frame):
    global texts
    for text in texts:
        text.remove()
    texts = []

    # step model once (your existing loop) ...
    for i in range(GSize):
        for j in range(GSize):
            cell_by_agent = []
            for a in agents_loc:
                d = np.hypot(a[0]-i, a[1]-j)
                s = cell_send_pulse(targets_loc, false_targets_loc, alpha, i, j)
                r = agent_recieved_signal(d, lamda) if s else 0
                cell_by_agent.append(cell_update_probabillity(grid[i][j], frame, d, r))
            update_cell(cell_by_agent, frame, grid, i, j)
            update_target_set(target_set, detect_img, i, j, threshold, grid[i][j], frame)

    # update heatmaps
    Gim.set_data(grid)
    Tim.set_data(detect_img)

    # annotate detections on Tax
    ii, jj = np.where(detect_img > 0)
    for y, x in zip(ii, jj):
        t = Tax.text(x, y, int(detect_img[y, x]), ha="center", va="center", color="black", fontsize=7)
        texts.append(t)

    # --- update bar "table" ---
    Bax.cla()
    Bax.set_title('Detections by cell')
    Bax.set_ylabel('Frame of first detection')
    Bax.set_ylim(0, nof_iterations)

    if ii.size:
        frames = detect_img[ii, jj]
        labels = [f"({i},{j})" for i, j in zip(ii, jj)]
        order = np.argsort(frames)  # sort by frame (earliest first)
        frames = frames[order]
        labels = [labels[k] for k in order]

        x = np.arange(len(frames))
        Bax.bar(x, frames)
        Bax.set_xticks(x)
        Bax.set_xticklabels(labels, rotation=90, fontsize=7)

    return [Gim, Tim, *texts]  # blit=False, so returning artists is optional

# ----- run -----
anim = animation.FuncAnimation(fig, update, frames=nof_iterations, interval=200, blit=False)
HTML(anim.to_jshtml())

# Multiple Runs, no simulation
>
this version gets the constant hyper-parameters(HP):
1. nof_runs (number of runs)
2. max_frames
3. gridsize
4. nof_agents
5. nof_targets
6. nof_false_targets

and the varying-HP:
1. alpha
2. lambda
3. threshold
>

It then iterates over the varying-HP and tries to find the {max-alpha, min-lambda, max-threshold} combination that would still find all targets withough finding false-targets.

In [ ]:
import pandas as pd

# List to store results from each run
results = []

nof_runs = 20
max_frames = 500
# Randomize hyperparameters (example ranges, adjust as needed)
current_alpha = 1.0
current_lamda = random.uniform(5, 20)
current_threshold = random.uniform(0.7, 0.95)
all_targets_detected = 1
reached_undetection = 5

for run in range(nof_runs):
    print(f"Starting run {run + 1}/{nof_runs} with alpha {current_alpha}")

    for i in range(reached_undetection) :
      choose_hyper = 0 #random.choice([0,1,2])
      match choose_hyper:
        case 0:   current_alpha += 0.05 if all_targets_detected else -0.05
        case 1:   current_lamda += -0.5 if all_targets_detected else 0.5
        case 2:   current_threshold += 0.01 if all_targets_detected else -0.01
        case _:   print('ERROR')

    # Reset variables for each run
    grid = np.full((GSize, GSize), nof_targets/(GSize**2), dtype=float)
    # Re-randomize agent, target, and false target locations for each run
    agents_loc = random.sample([(i,j) for i in range(GSize) for j in range(GSize)], nof_agents)
    targets_loc = random.sample([(i,j) for i in range(GSize) for j in range(GSize)], nof_targets)
    available_locs = [(i,j) for i in range(GSize) for j in range(GSize) if (i,j) not in targets_loc]
    false_targets_loc = random.sample(available_locs, nof_false_targets)

    detect_img = np.zeros_like(grid, dtype=float)
    target_set = set()
    frames_taken = 0
    false_target_detected = False # Flag to indicate if a false target was detected

    # --- Simulation loop for a single run ---
    for frame in range(max_frames):
        # Step model once (copying logic from the previous update function)
        for i in range(GSize):
            for j in range(GSize):
                cell_by_agent = []
                for a in agents_loc:
                    d = np.hypot(a[0]-i, a[1]-j)
                    s = cell_send_pulse(targets_loc, false_targets_loc, current_alpha, i, j) # Use current_alpha
                    r = agent_recieved_signal(d, current_lamda) if s else 0 # Use current_lamda
                    cell_by_agent.append(cell_update_probabillity(grid[i][j], frame, d, r))
                update_cell(cell_by_agent,frame, grid, i, j)
                update_target_set(target_set, detect_img, i, j, current_threshold, grid[i][j], frame) # Use current_threshold

        frames_taken = frame + 1

        # Check stopping condition
        detected_target_locations = [(t[0], t[1]) for t in target_set]
        if all(target in detected_target_locations for target in targets_loc):
            print(f"All targets detected in run {run + 1} at frame {frames_taken}")
            break

        # Check if any detected location is a false target
        if any(detected_loc in false_targets_loc for detected_loc in detected_target_locations):
            false_target_detected = True
            print(f"False Target detected in run {run + 1} at frame {frames_taken}")
            break

    # --- Collect results for the run ---
    last_detected_target_frame = -1
    last_detected_target_loc = None
    if target_set:
        # Find the target that was discovered last
        # Filter out false targets before finding the last detected true target
        true_targets_detected = [t for t in target_set if (t[0], t[1]) in targets_loc]
        if true_targets_detected:
          last_detected_target_info = max(true_targets_detected, key=lambda item: item[2])
          last_detected_target_loc = (last_detected_target_info[0], last_detected_target_info[1])
          last_detected_target_frame = last_detected_target_info[2]


    avg_agent_distance_to_last_target = -1
    if last_detected_target_loc:
        distances = [np.hypot(a[0] - last_detected_target_loc[0], a[1] - last_detected_target_loc[1]) for a in agents_loc]
        avg_agent_distance_to_last_target = np.mean(distances)

    all_targets_detected = all(target in detected_target_locations for target in targets_loc)
    if all_targets_detected == 1 : reached_undetection = 1

    results.append({
        'run': run + 1,
        'nof_agents': nof_agents,
        'nof_targets': nof_targets,
        'alpha': current_alpha,
        'lamda': current_lamda,
        'threshold': current_threshold,
        'frames_to_detect_all': frames_taken if all_targets_detected else -1, # -1 if not all detected
        'avg_agent_distance_to_last_target': avg_agent_distance_to_last_target,
        'false_target_detected': false_target_detected # Add the false target detection flag
    })

# Convert results to a pandas DataFrame
results_df = pd.DataFrame(results)

# Display the results
display(results_df)